In [1]:
from flask import Flask, request, jsonify
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
import uuid
import transformers
 
app = Flask(__name__)
 
USE_PEFT_BACKEND = False
 
# Check if PEFT backend is available
if hasattr(transformers, "pipelines"):
    if hasattr(transformers.pipelines, "PEFTBackend"):
        USE_PEFT_BACKEND = True
 
# Initialize the DiffusionPipeline with the pre-trained model
pipeline = DiffusionPipeline.from_pretrained("fluently/Fluently-XL-v2")
pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeline.scheduler.config)
 
# Load the weights for DALL·E model
if USE_PEFT_BACKEND:
    pipeline.load_lora_weights("ehristoforu/dalle-3-xl-v2", weight_name="dalle-3-xl-lora-v2.safetensors", adapter_name="dalle")
 
def save_image(img):
    unique_name = str(uuid.uuid4()) + ".png"
    img.save(unique_name)
    return unique_name
 
@app.route('/generate_image', methods=['POST'])
def generate_image():
    data = request.json
    text_prompt = data.get('text_prompt', '')
 
    # Generate the image based on the textual prompt
    images = pipeline(
            prompt=text_prompt,
            negative_prompt="",
            width=512,
            height=512,
            guidance_scale=3,
            num_inference_steps=15,
            num_images_per_prompt=1,
            cross_attention_kwargs={"scale": 0.70},
            output_type="pil",
        ).images
   
    image_paths = [save_image(img) for img in images]
   
    return jsonify({"image_paths": image_paths})
 
if __name__ == '__main__':
    app.run(debug=True)

c:\Users\GAUTA\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 7/7 [00:24<00:00,  3.52s/it]


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\GAUTA\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:

from langchain.llms import CTransformers
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import PromptTemplate
 
# Define the template for the prompt
prompt_template_twitter = """Generate a list of two varied versions of Twitter post sequences. \
    The topic of the post is as follows:
   
    current conversation:{history}
    Human: {input}
   
 
    You are required to write it in English. Keep it fun to read by adding some emojis and supporting hashtags (just if you think it's necessary).
 
    Output it as an array with  2 JSON items format with the following keys:
    - version: <version 1/2>,
    - tweet: <the tweet, each thread separated by the number of the sequence and new line char>
    """
 
# Function to get user input for the prompt
def get_user_prompt():
    query = input("Enter your query: ")
    return query
 
# Instantiate the PromptTemplate with the template and input variables
# prompt = PromptTemplate(template=prompt_template, input_variables=['query'])
prompt_twitter = PromptTemplate( input_variables=['history','input'],template=prompt_template_twitter)
# Instantiate the CTransformers model
llm = CTransformers(model="./mistral-7b-instruct-v0.2.Q4_K_M.gguf",
                    model_type="llama",
                    max_new_tokens=4000 ,
                    temperature=0.2)
 
# Instantiate the LLMChain with the prompt and llm
# llm_chain = LLMChain(prompt=prompt, llm=llm)
conversation=ConversationChain(
    prompt=prompt_twitter,
    llm=llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=2)
   
)
# Define a function to handle user interaction
def main():
    while True:
        # Get user input for the prompt
        user_prompt = get_user_prompt()
 
        # Run the LLMChain with the user input prompt
        llm_response = conversation.run(input= user_prompt)
 
        # Print the response
        print(llm_response)
 
        # Ask if the user wants to ask another question
        another_question = input("Do you want to ask another question? (yes/no): ")
        if another_question.lower() != 'yes':
            break
 
# Call the main function to start the interaction
main()
 

In [1]:
!pip install langchain

In [2]:
!pip install CTransformers

In [3]:
from langchain.llms import CTransformers
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import PromptTemplate

# Define the template for the proposal prompt
prompt_template_proposal = """Generate a list of two varied versions of proposal sequences. The topic of the proposal is as follows:

current conversation: {history}
Human: {input}

You are required to write it in a formal but engaging tone. Make sure to include all necessary details and sections in the proposal.

Output it as an array with 2 JSON items format with the following keys:
- version: <version 1/2>,
- proposal: <the proposal, each section separated by the name of the section and a new line char>
"""

# Function to get user input for the proposal prompt
def get_user_prompt():
    query = input("Enter your proposal prompt: ")
    return query

# Instantiate the PromptTemplate with the proposal template and input variables
prompt_proposal = PromptTemplate(input_variables=['history', 'input'], template=prompt_template_proposal)

# Instantiate the CTransformers model
llm = CTransformers(model="./mistral-7b-instruct-v0.2.Q4_K_M.gguf",
                    model_type="llama",
                    max_new_tokens=4000,
                    temperature=0.2)

# Instantiate the ConversationChain with the proposal prompt and llm
conversation = ConversationChain(
    prompt=prompt_proposal,
    llm=llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=2)
)

# Define a function to handle user interaction
def main():
    while True:
        # Get user input for the proposal prompt
        user_prompt = get_user_prompt()

        # Run the ConversationChain with the user input prompt
        llm_response = conversation.run(input=user_prompt)

        # Print the response
        print(llm_response)

        # Ask if the user wants to ask another question
        another_question = input("Do you want to generate another proposal? (yes/no): ")
        if another_question.lower() != 'yes':
            break

# Call the main function to start the interaction
main()

C:\Users\GAUTA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\GAUTA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
Generate a list of two varied versions of proposal sequences. The topic of the proposal is as follows:

current conversation: 
Human: Generate a proposal for Convince the client of the return on investment (ROI) and long-term benefits of our proposed solution.

You are required to write it in a formal but engaging tone. Make sure to include all necessary details and sections in the proposal.

Output it as an array with 2 JSON items format with the following keys:
- version: <version 1/2>,
- proposal: <the proposal, each section separated by the name of the section and a new line char>


> Finished chain.

---
Version 1:
[
  {
    "version": "Version I: Value-Driven ROI Proposal",
    "proposal": "**Executive Summary:**\nOur proposed solution offers significant value to your business. The short-term benefits include increased productivity and efficiency, while the long-term advantages encompass cost savings, improved c